In [2]:
import warnings 
warnings.filterwarnings("ignore")
import numpy 
import cv2 

import PIL.Image as Image
import os

%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf 

CLASSSES = ['cat', 'dog']
LEARNING_RATE = 0.001
EPOCHS = 500

2022-03-07 14:26:27.480950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zalven/anaconda3/envs/Deeplearning/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-07 14:26:27.481054: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<img src='./image_net_architecture.jpeg' width='300px'>

In [4]:
def depth_block(layer, strides):
    layer = tf.keras.layers.DepthwiseConv2D(3, strides=strides, padding='same', use_bias=False)(layer)
    layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.ReLU()(layer)
    return layer 

def single_conv_block(layer,filters):
    layer = tf.keras.layers.Conv2D(filters, 1,use_bias=False)(layer)
    layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.ReLU()(layer)
    return layer

def combo_layer(layer,filters,strides):
    layer = depth_block(layer,strides=strides)
    layer = single_conv_block(layer, filters)
    return layer 

    
def MobileNet(input_shape=(224,224,3),n_classes = 1000):
    model_input = tf.keras.Input(input_shape)
    layer = tf.keras.layers.Conv2D(32,3, strides=(2,2), padding='same', use_bias=False)(model_input)
    layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.ReLU()(layer)
    layer = combo_layer(layer,64, strides=(2,2))
    layer  = combo_layer(layer ,128,strides=(2,2))
    layer  = combo_layer(layer ,128,strides=(1,1))
    layer = combo_layer(layer,256,strides=(2,2))
    layer = combo_layer(layer,256,strides=(1,1))
    layer = combo_layer(layer,512,strides=(2,2))
    for _ in range(5):
        layer  = combo_layer(layer ,512,strides=(1,1))
    layer = combo_layer(layer,1024,strides=(2,2))
    layer = combo_layer(layer,1024,strides=(1,1))
    layer = tf.keras.layers.GlobalAveragePooling2D()(layer)
    model_output = tf.keras.layers.Dense(n_classes, activation=tf.keras.activations.softmax)(layer)
    model = tf.keras.Model(model_input,model_output)

    return model 


input_shape = (224,224,3)
model = MobileNet(input_shape,len(CLASSSES))
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)
# history = model.fit(epochs=EPOCHS)